# Informe Final de la Prueba Técnica 2

**Autor:** Isabel Castrillón Acosta

**Fecha:** 25-06-2025

**Descripción:** Este notebook presenta un informe detallado de la prueba técnica 2, que incluye la implementación de un chatbot legal utilizando modelos de lenguaje de Open AI. 



# 1. Explicación del Caso

# 2. Supuestos

# 3. Formas de resolver el caso y la opcion elegida

# 4. Resultados del análisis de los datos y los modelos

## 4.1 Librerias

In [1]:
import pandas as pd
import faiss
import numpy as np
import os
from openai import OpenAI
from sentence_transformers import SentenceTransformer
from dotenv import load_dotenv


## 4.2 Carga de datos


In [2]:
df = pd.read_excel('C:/Users/Isabel/Documents/Dataknow-PT/Prueba tecnica 2/Datos/sentencias_pasadas.xlsx')
df.head()

,#,Relevancia,Providencia,Tipo,Fecha Sentencia,Tema - subtema,resuelve,sintesis
0,1,966965.0,T-185/22,NaN,2022-05-31,NaN,en nombre del pueblo y por mandato de la Const...,En este caso se formula la acción de tutela en...
1,3,963168.0,T-356/21,NaN,2021-10-15,ACCIÓN DE TUTELA PARA PROTEGER EL DERECHO A LA...,en nombre del pueblo y por mandato de la Const...,El peticionario considera que los accionantes ...
2,5,956201.0,T-351/22,NaN,2022-10-07,ACCIÓN DE TUTELA PARA PROTEGER EL DERECHO A LA...,"administrando justicia en nombre del Pueblo, y...",El periodista accionante acusa al abogado acci...
3,6,955889.0,T-246/21,NaN,2021-07-29,ACCION DE TUTELA PARA PROTEGER EL DERECHO A LA...,en nombre del pueblo y por mandato de la Const...,Se presenta la acción de tutela en contra de u...
4,7,955787.0,T-245A/22,NaN,2022-07-01,ACCION DE TUTELA-Inexistencia de hecho superad...,en nombre del pueblo y por mandato de la Const...,"El accionante, actuando en representación de s..."


## 4.3 Preparación de los datos


In [3]:
df['texto'] = df[['Providencia', 'Tipo', 'Fecha Sentencia', 'Tema - subtema', 'resuelve', 'sintesis']].astype(str).agg(' '.join, axis=1)
df['texto'].iloc[0]

'T-185/22 nan 2022-05-31 nan en nombre del pueblo y por mandato de la Constitución, RESUELVE PRIMERO.- REVOCAR las sentencias expedidas el 6 de enero de 2021, por el Juzgado Veinte Penal Municipal con Función de Control de Garantías de Bogotá, en primera instancia, y el 12 de febrero de 2021, por el Juzgado Cincuenta Penal del Circuito con Función de Conocimiento de Bogotá, en segunda instancia, mediante las cuales concedieron el amparo de los derechos fundamentales al buen nombre y a la honra de Enrique Vallejo Echeverri, Paula Camila Fonseca y la Clínica Envalle S.A.S. En su lugar, DECLARAR IMPROCEDENTE el amparo invocado por los accionantes, por las razones expuestas en la parte motiva de esta providencia. SEGUNDO.- Por Secretaría General, LÍBRENSE las comunicaciones de que trata el artículo 36 del Decreto 2591 de 1991. En este caso se formula la acción de tutela en contra de un particular, alegando la vulneración de los derechos fundamentales al buen nombre y honra, en virtud de la

## 4.4 Vectorización de los datos


Convertir los textos en embeddings:

In [4]:
model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = model.encode(df['texto'].tolist(), convert_to_numpy=True)

Guardar en base de datos vectorial con librería FAISS:


In [5]:
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

## 4.5 Función de búsqueda

In [6]:
def buscar_casos(query, top_k=3):
    query_emb = model.encode([query])
    D, I = index.search(query_emb, top_k)
    resultados = df.iloc[I[0]][['Providencia', 'texto']]
    return resultados


In [7]:
load_dotenv()
client = OpenAI(api_key= os.getenv('OPENAI_API_KEY'))  

def consulta_chatbot_openai(pregunta, top_k=3):
    contexto = buscar_casos(pregunta, top_k=top_k).head(3)
    contexto_completo = ""
    for i, fila in contexto.iterrows():
        contexto_completo += f"Caso {i+1}:\n{fila['texto']}\n\n"

    prompt = f"""Actúa como un asesor legal que explica de manera sencilla para alguien sin conocimientos jurídicos.

Contexto extraído de casos reales:
{contexto_completo}

Pregunta: {pregunta}
Respuesta:"""

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.5,
        max_tokens=500
    )

    return response.choices[0].message.content.strip()



In [8]:
pregunta = "¿Cuál fue la sentencia del caso de acoso escolar?"
respuesta = consulta_chatbot_openai(pregunta)
print(respuesta)

En el caso de acoso escolar (Caso 68), la sentencia fue la revocación parcial de la sentencia de instancia que negó el amparo solicitado. Se declaró la carencia actual de objeto por daño consumado, pero se concluyó que la entidad educativa vulneró garantías constitucionales del menor representado al no garantizarle un acompañamiento idóneo ni activar las rutas de atención correspondientes contra el acoso escolar que sufrió. Se impartieron órdenes a la entidad para que se abstenga de dilaciones injustificadas frente a casos de acoso escolar, incluya en sus protocolos de atención mecanismos de capacitación sobre acoso escolar, ciberbullying y otros temas, y establezca medidas de reparación integral y garantías de no repetición a las víctimas de acoso escolar.


# 5. Futuras mejoras

# 6. Apreciaciones y comentarios del caso (opcional)